In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from varclushi import VarClusHi
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_excel('Data- acquisition scorecard.xlsx')

In [3]:
df.rename(columns={'ENQ_1':'days_last_enq','ENQ_2':'days_hl_last_enq','ENQ_3':'#_enq_3mnths','ENQ_4':'#_hle_enq_3mnths','ENQ_5':'#_enq_12mnths','ENQ_6':'#_hle_enq_12mnths','ENQ_7':'Tot_enq','ENQ_8':'Tot_hle_enq','ACCOUNT_9':'#_defaults_3mnths','ACCOUNT_10':'#_defaults_12mnths','ACCOUNT_11':'days_last_acc_open','ACCOUNT_12':'Tot_accounts','ACCOUNT_13':'Tot_hle_accounts','ACCOUNT_14':'Tot_unsecured_loans','ACCOUNT_15':'Tot_live_loans','ACCOUNT_16':'Tot_live_homeloans','ACCOUNT_17':'Tot_live_unsecuredloans','ACCOUNT_18':'Tot_amt_outstanding','ACCOUNT_19':'Tot_secured_outstanding_amt','ACCOUNT_20':'Tot_unsecured_outstanding_amt','ACCOUNT_21':'Avg_days_in_debt'},inplace=True)

In [4]:
df_copy = df.copy()

In [5]:
df = df.loc[:,['ID','Tot_enq','Tot_unsecured_outstanding_amt','Target']]

In [6]:
df.shape

(3980, 4)

In [7]:
print(df.shape)
df.drop_duplicates(inplace = True)
print(df.shape)

(3980, 4)
(3894, 4)


In [8]:
df = df.set_index('ID')

In [9]:
df.head()

,Tot_enq,Tot_unsecured_outstanding_amt,Target
ID,,,
1794,14,275606,0
3183,11,20,0
1841,3,0,0
1115,3,0,0
3125,2,0,0


In [10]:
df['account20_bkt'] = pd.cut( df['Tot_unsecured_outstanding_amt'] , [-np.inf , 4500, 160000, 455000, np.inf ])
df['enq7_bkt']      = pd.cut( df['Tot_enq'] , [-np.inf, 3, 7, 14, 22, np.inf])

In [20]:
lst = pd.qcut(pred_numeric[:,1],10)

In [21]:
df['prob_num'] = lst

In [22]:
df.head()

,Tot_enq,Tot_unsecured_outstanding_amt,Target,account20_bkt,enq7_bkt,prob_num
ID,,,,,,
1794,14,275606,0,"(160000.0, 455000.0]","(7.0, 14.0]","(0.0445, 0.138]"
3183,11,20,0,"(-inf, 4500.0]","(7.0, 14.0]","(0.138, 0.198]"
1841,3,0,0,"(-inf, 4500.0]","(-inf, 3.0]","(0.36, 0.392]"
1115,3,0,0,"(-inf, 4500.0]","(-inf, 3.0]","(0.36, 0.392]"
3125,2,0,0,"(-inf, 4500.0]","(-inf, 3.0]","(0.392, 0.426]"


In [23]:
t = pd.pivot_table( df, index = 'prob_num', columns = 'Target', values = 'Tot_enq', aggfunc = 'count')
t

Target,0,1
prob_num,,
"(-0.0009999999941700001, 0.0445]",322,68
"(0.0445, 0.138]",365,43
"(0.138, 0.198]",345,25
"(0.198, 0.27]",390,34
"(0.27, 0.305]",331,24
"(0.305, 0.36]",459,22
"(0.36, 0.392]",365,25
"(0.392, 0.426]",418,20
"(0.426, 0.461]",360,17


In [12]:
x = df.drop(columns = ['Target','account20_bkt','enq7_bkt'])
y = df.Target

In [13]:
model_with_numeric = LogisticRegression()

In [14]:
model_with_numeric.fit(x,y)

LogisticRegression()

In [15]:
pred_numeric = model_with_numeric.predict_proba(x)

In [16]:
pred_numeric

array([[0.87334804, 0.12665196],
       [0.82570394, 0.17429606],
       [0.60769057, 0.39230943],
       ...,
       [0.70199497, 0.29800503],
       [0.5739245 , 0.4260755 ],
       [0.78177099, 0.21822901]])

In [17]:
x.head()

,Tot_enq,Tot_unsecured_outstanding_amt
ID,,
1794,14,275606
3183,11,20
1841,3,0
1115,3,0
3125,2,0


In [18]:
y.head()

ID
1794    0
3183    0
1841    0
1115    0
3125    0
Name: Target, dtype: int64

In [19]:
roc_auc_score(y,pred_numeric[:,1])

0.38707157841300177

In [37]:
df.head()

,Tot_enq,Tot_unsecured_outstanding_amt,Target,account20_bkt,enq7_bkt
ID,,,,,
1794,14,275606,0,"(160000.0, 455000.0]","(7.0, 14.0]"
3183,11,20,0,"(-inf, 4500.0]","(7.0, 14.0]"
1841,3,0,0,"(-inf, 4500.0]","(-inf, 3.0]"
1115,3,0,0,"(-inf, 4500.0]","(-inf, 3.0]"
3125,2,0,0,"(-inf, 4500.0]","(-inf, 3.0]"


In [24]:
x = df.drop(columns = ['Target','Tot_unsecured_outstanding_amt','Tot_enq'])
y = df.Target

In [25]:
x = pd.get_dummies(x)

In [26]:
model_with_bins = LogisticRegression()

In [27]:
model_with_bins.fit(x,y)

LogisticRegression()

In [28]:
pred_binned = model_with_bins.predict_proba(x)

In [29]:
roc_auc_score(y,pred_binned[:,1])

0.6425549268015917

In [31]:
df.head()

,Tot_enq,Tot_unsecured_outstanding_amt,Target,account20_bkt,enq7_bkt,prob_num
ID,,,,,,
1794,14,275606,0,"(160000.0, 455000.0]","(7.0, 14.0]","(0.0445, 0.138]"
3183,11,20,0,"(-inf, 4500.0]","(7.0, 14.0]","(0.138, 0.198]"
1841,3,0,0,"(-inf, 4500.0]","(-inf, 3.0]","(0.36, 0.392]"
1115,3,0,0,"(-inf, 4500.0]","(-inf, 3.0]","(0.36, 0.392]"
3125,2,0,0,"(-inf, 4500.0]","(-inf, 3.0]","(0.392, 0.426]"


In [26]:
t = pd.pivot_table( df, index = 'account20_bkt', columns = 'Target', values = 'Tot_enq', aggfunc = 'count')
t

Target,0,1
account20_bkt,,
"(-inf, 4500.0]",1826,121
"(4500.0, 160000.0]",1073,81
"(160000.0, 455000.0]",364,38
"(455000.0, inf]",333,58


In [27]:
t['ba_rate'] = t[1]/(t[0]+t[1])*100
t

Target,0,1,ba_rate
account20_bkt,,,
"(-inf, 4500.0]",1826,121,6.215
"(4500.0, 160000.0]",1073,81,7.019
"(160000.0, 455000.0]",364,38,9.453
"(455000.0, inf]",333,58,14.834


In [26]:
t = pd.pivot_table( df, index = 'account20_bkt', columns = 'Target', values = 'Tot_enq', aggfunc = 'count')
t

Target,0,1
account20_bkt,,
"(-inf, 4500.0]",1826,121
"(4500.0, 160000.0]",1073,81
"(160000.0, 455000.0]",364,38
"(455000.0, inf]",333,58


In [32]:
pred_binned

array([[0.85574169, 0.14425831],
       [0.93283608, 0.06716392],
       [0.93800388, 0.06199612],
       ...,
       [0.93566027, 0.06433973],
       [0.95420893, 0.04579107],
       [0.91437164, 0.08562836]])

In [38]:
lst = pd.qcut(pred_binned[:,1],6)

In [39]:
df['prob_num'] = lst

In [41]:
t = pd.pivot_table( df, index = 'prob_num', columns = 'Target', values = 'Tot_enq', aggfunc = 'count')
t

Target,0,1
prob_num,,
"(0.0411, 0.0451]",651,30
"(0.0451, 0.0503]",612,27
"(0.0503, 0.063]",611,42
"(0.063, 0.0737]",600,42
"(0.0737, 0.0976]",611,64
"(0.0976, 0.21]",511,93


In [46]:
t = pd.pivot_table( df, index = 'prob_num', columns = 'Target', values = 'Tot_enq', aggfunc = 'count')
t

Target,0,1
prob_num,,
"(-0.0009999999941700001, 0.0445]",322,68
"(0.0445, 0.138]",365,43
"(0.138, 0.198]",345,25
"(0.198, 0.27]",390,34
"(0.27, 0.305]",331,24
"(0.305, 0.36]",459,22
"(0.36, 0.392]",365,25
"(0.392, 0.426]",418,20
"(0.426, 0.461]",360,17
